# Creation of Intents

#### Loading Data

In [119]:
import pandas as pd

In [120]:
pd.set_option('display.max_colwidth', None)

In [121]:
data = pd.read_csv("question_responce.csv").drop(columns='Unnamed: 0')
data = data.drop_duplicates()
data.head(5)

,responce,question
0,@115904 We'll be sure to pass along your kind words! #AATeam,@AmericanAir Erica on the lax team is amazing give her a raise ty
1,@115904 Our apologies for the delay in responding to you. Have you made it to LAX? Let us know if you still need assistance.,@AmericanAir Could you have someone on your lax team available to guide me to my gate ASAP
2,"@115905 Aww, that's definitely a future pilot in the making! #HappyHalloween",Ben Tennyson and an American Airlines pilot. 🎃 #trunkortreat #halloween #2017 #diycostume #parenting @americanair … https://t.co/f1nNHQ0iLa https://t.co/lDViDkRdB1
3,@115906 We're sorry for your frustration.,"@AmericanAir Right, but I earned those. I also shouldn’t have to pay to pass them to my own spouse. You need to change your program."
4,@115909 We're glad you got to kick back and enjoy a show while flying! Thanks for your kind words.,"Thank you, @AmericanAir for playing #ThisIsUs and for having great flight attendants on my flight back home!"


#### Preprocessing 

**Steps :**
- All words starting by '@' and '#' are removed
- Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
- All stopwords are removed.
- Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
- bags of words are created

In [122]:
import gensim
import nltk
import numpy as np
import json
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

np.random.seed(2018)
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('words')
words = set(nltk.corpus.words.words())

Extra stopwords

In [123]:
extra_words = ['flight','flights','fly','flying','plane','thanks', 'thank','ty','get','please','plz']
stop_words = stopwords.words('english')
stop_words.extend(extra_words)

In [124]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

#### Preprocessing

Remove unecessary items int the text

In [125]:
def question_preprocess(text):
    text = " ".join(filter(lambda x:x[0]!='@', text.split()))
    text = " ".join(filter(lambda x:x[0]!='#', text.split()))
    text = " ".join(filter(lambda x:x[0:4]!='http', text.split()))
    return text

In [126]:
data['question_clean'] = data['question'].apply(question_preprocess)
data['question_clean']

0                                                                          Erica on the lax team is amazing give her a raise ty
1                                                 Could you have someone on your lax team available to guide me to my gate ASAP
2                                                                              Ben Tennyson and an American Airlines pilot. 🎃 …
3       Right, but I earned those. I also shouldn’t have to pay to pass them to my own spouse. You need to change your program.
4                                         Thank you, for playing and for having great flight attendants on my flight back home!
                                                                 ...                                                           
1847                                and have nailed in the transatlantic WiFi service. I am able to join my daily scrum onboard
1848                                                                                Average price of tic

In [127]:
data['question_preproc'] = data['question_clean'].apply(gensim.utils.simple_preprocess)
data['question_preproc']

0                                                                             [erica, on, the, lax, team, is, amazing, give, her, raise, ty]
1                                             [could, you, have, someone, on, your, lax, team, available, to, guide, me, to, my, gate, asap]
2                                                                                        [ben, tennyson, and, an, american, airlines, pilot]
3       [right, but, earned, those, also, shouldn, have, to, pay, to, pass, them, to, my, own, spouse, you, need, to, change, your, program]
4                                        [thank, you, for, playing, and, for, having, great, flight, attendants, on, my, flight, back, home]
                                                                        ...                                                                 
1847                               [and, have, nailed, in, the, transatlantic, wifi, service, am, able, to, join, my, daily, scrum, onboard]
1848         

In [128]:
def remove_stopwords(text):
    return [[w for w in simple_preprocess(str(doc)) if w not in stop_words] for doc in text]

In [129]:
data["question_stop"] = remove_stopwords(data['question_preproc'])
data["question_stop"] = data["question_stop"].apply(" ".join)
data["question_stop"]

0                                                   erica lax team amazing give raise
1                                    could someone lax team available guide gate asap
2                                                ben tennyson american airlines pilot
3                               right earned also pay pass spouse need change program
4                                                  playing great attendants back home
                                            ...                                      
1847                  nailed transatlantic wifi service able join daily scrum onboard
1848                                                     average price ticket one way
1849          really annoyed month since damaged bag claim never heard back done told
1850    terrible service wait ages trying call number almost two months gone response
1851                                 charges patrons change every time airport closes
Name: question_stop, Length: 1852, dtype: object

Lemmatize

In [130]:
def lemmat(text):
    result = []
    text
    for token in gensim.utils.simple_preprocess(text):
        if token not in stop_words and len(token) > 2:
            result.append(WordNetLemmatizer().lemmatize(token))
    return result


In [131]:
data ["question_lem"] = data["question_stop"].apply(lemmat)

In [132]:
data[["question","question_lem"]]

,question,question_lem
0,@AmericanAir Erica on the lax team is amazing give her a raise ty,"[erica, lax, team, amazing, give, raise]"
1,@AmericanAir Could you have someone on your lax team available to guide me to my gate ASAP,"[could, someone, lax, team, available, guide, gate, asap]"
2,Ben Tennyson and an American Airlines pilot. 🎃 #trunkortreat #halloween #2017 #diycostume #parenting @americanair … https://t.co/f1nNHQ0iLa https://t.co/lDViDkRdB1,"[ben, tennyson, american, airline, pilot]"
3,"@AmericanAir Right, but I earned those. I also shouldn’t have to pay to pass them to my own spouse. You need to change your program.","[right, earned, also, pay, pas, spouse, need, change, program]"
4,"Thank you, @AmericanAir for playing #ThisIsUs and for having great flight attendants on my flight back home!","[playing, great, attendant, back, home]"
...,...,...
1847,@AmericanAir and @172 have nailed in the transatlantic WiFi service. I am able to join my @172377 daily scrum onboard,"[nailed, transatlantic, wifi, service, able, join, daily, scrum, onboard]"
1848,@AmericanAir Average price of ticket out: $2500 one way.,"[average, price, ticket, one, way]"
1849,@AmericanAir Really annoyed been over a month since my damaged bag claim never heard back! done as told...,"[really, annoyed, month, since, damaged, bag, claim, never, heard, back, done, told]"
1850,@AmericanAir terrible service wait ages trying to call that number almost two months gone no response,"[terrible, service, wait, age, trying, call, number, almost, two, month, gone, response]"


#### Making bigrams and trigrams

In [133]:
bigram = gensim.models.Phrases(data["question_lem"], min_count=3, threshold=10)
bigram_mod = gensim.models.phrases.Phraser(bigram)

#example
for i in range(15,24):
    print(bigram_mod[data["question_lem"][i]])

# we obtain coherent bigrams as customer_service, business_class, etc..

['god', 'going', 'ten', 'minute']
['give', 'grief', 'thing', 'poorly', 'credit', 'guacamole', 'admiral_club', 'outstanding', 'addition']
['aware', 'fit', 'overhead', 'return_trip', 'luggage', 'going', 'pre', 'security', 'gate', 'chck', 'empty', 'overhead', 'flt']
['despite', 'ewr', 'forced', 'mia', 'check', 'guitar', 'amp', 'charged', 'gate', 'security', 'flt', 'full']
['fifteen', 'empty', 'business_class', 'seat', 'one', 'person', 'got', 'upgraded']
['still_waiting', 'compensate', 'crap', 'put', 'honeymoon', 'almost', 'year', 'ago']
['really', 'hour', 'international', 'seat', 'back', 'entertainment', 'screen', 'mon']
['still', 'rather', 'screen', 'though', 'rather', 'cause', 'neck', 'problem', 'looking', 'tablet']
['say', 'attenants', 'bdl', 'ord', 'awesome', 'gate_agent', 'bdl', 'also', 'need', 'shout', 'awesome', 'customer_service', 'treated', 'nonrev', 'well']


In [134]:
def bigrams(text):
    return [bigram_mod[doc] for doc in text]

In [135]:
data["question_lem_bigrams"] = bigrams(data["question_lem"])

In [136]:
data = data[['responce', 'question','question_lem_bigrams']]

#### Building LDA

In [137]:
import gensim.corpora as corpora

In [138]:
id2word = corpora.Dictionary(data["question_lem_bigrams"])
id2word.filter_extremes(no_below=3)
corpus = [id2word.doc2bow(text) for text in data["question_lem_bigrams"]]

In [139]:
lda_model_10 = gensim.models.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10,
                                       update_every=1,
                                       chunksize=100,
                                       passes=10,
                                       alpha='auto',
                                       per_word_topics=True)

In [140]:
lda_model_20 = gensim.models.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=20,
                                       update_every=1,
                                       chunksize=100,
                                       passes=10,
                                       alpha='auto',
                                       per_word_topics=True)

In [142]:
lda_model_30 = gensim.models.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=40,
                                       update_every=1,
                                       chunksize=100,
                                       passes=10,
                                       alpha='auto',
                                       per_word_topics=True)

In [143]:
lda_model_40 = gensim.models.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=40,
                                       update_every=1,
                                       chunksize=100,
                                       passes=10,
                                       alpha='auto',
                                       per_word_topics=True)

In [144]:
lda_model_80 = gensim.models.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=80,
                                       update_every=1,
                                       chunksize=100,
                                       passes=10,
                                       alpha='auto',
                                       per_word_topics=True)

In [145]:
print("lda_model_10: ",lda_model_10[corpus[0]])
print("lda_model_20: ",lda_model_20[corpus[0]])
print("lda_model_30: ",lda_model_30[corpus[0]])
print("lda_model_40: ",lda_model_40[corpus[0]])
print("lda_model_80: ",lda_model_80[corpus[0]])

lda_model_10:  ([(0, 0.07004369), (1, 0.096937574), (2, 0.16898799), (3, 0.0707583), (4, 0.06325248), (5, 0.1362259), (6, 0.12934561), (7, 0.041932203), (8, 0.066110216), (9, 0.15640602)], [(0, [6]), (1, [5]), (2, [9]), (3, [2])], [(0, [(6, 0.99992937)]), (1, [(5, 0.99993473)]), (2, [(9, 0.9998792)]), (3, [(2, 0.99991477)])])
lda_model_20:  ([(0, 0.021963846), (1, 0.025180485), (2, 0.018287798), (3, 0.028285634), (4, 0.02625115), (5, 0.018549582), (6, 0.03345007), (7, 0.12411707), (8, 0.13964856), (9, 0.021540275), (10, 0.042577304), (11, 0.12924385), (13, 0.034634907), (14, 0.031742483), (15, 0.0489589), (16, 0.038646106), (17, 0.1293983), (18, 0.04438643), (19, 0.038268093)], [(0, [8]), (1, [11]), (2, [17]), (3, [7])], [(0, [(8, 0.9999711)]), (1, [(11, 0.9999792)]), (2, [(17, 0.9999618)]), (3, [(7, 0.99997526)])])
lda_model_30:  ([(1, 0.03488962), (2, 0.03037113), (4, 0.028273873), (5, 0.014021162), (6, 0.012112436), (8, 0.03060544), (9, 0.0109267915), (10, 0.017701084), (11, 0.02755

our example corpus[0] is allegedly well classified in a topic by ~80%. but sometimes the lda_models hesitates between topics

display the topic's top 10 words

In [146]:
#example with 20 topics
topwords = np.array(lda_model_20.show_topics(num_topics=80, num_words=10))
for i in enumerate(topwords[:,1]):
    print(i)

(0, '0.187*"way" + 0.105*"nothing" + 0.089*"said" + 0.082*"since" + 0.072*"tomorrow" + 0.055*"staff" + 0.053*"job" + 0.050*"better" + 0.031*"reservation" + 0.021*"window"')
(1, '0.118*"home" + 0.093*"attendant" + 0.092*"call" + 0.073*"gate_agent" + 0.061*"number" + 0.053*"email" + 0.052*"phone" + 0.045*"americanair" + 0.044*"making" + 0.042*"helpful"')
(2, '0.279*"still" + 0.100*"much" + 0.095*"booked" + 0.067*"charged" + 0.040*"fare" + 0.040*"complimentary" + 0.039*"finally" + 0.033*"available" + 0.031*"san" + 0.015*"wonderful"')
(3, '0.147*"airline" + 0.079*"happy" + 0.075*"take" + 0.073*"could" + 0.060*"luggage" + 0.057*"charge" + 0.055*"carry" + 0.046*"believe" + 0.044*"ridiculous" + 0.040*"disappointed"')
(4, '0.180*"one" + 0.138*"pay" + 0.099*"yes" + 0.085*"trying" + 0.073*"made" + 0.052*"app" + 0.040*"booking" + 0.036*"night" + 0.032*"person" + 0.028*"dallas"')
(5, '0.206*"need" + 0.160*"never" + 0.104*"broken" + 0.059*"book" + 0.048*"plane" + 0.045*"small" + 0.034*"suitcase" + 

#### pyLDAvis: Interactive Topic Modelling Exploration

In [147]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [148]:
pyLDAvis.enable_notebook()
visual_10 = pyLDAvis.gensim.prepare(lda_model_10, corpus, id2word)
visual_20 = pyLDAvis.gensim.prepare(lda_model_20, corpus, id2word)
visual_30 = pyLDAvis.gensim.prepare(lda_model_30, corpus, id2word)
visual_40 = pyLDAvis.gensim.prepare(lda_model_40, corpus, id2word)
visual_80 = pyLDAvis.gensim.prepare(lda_model_80, corpus, id2word)

In [149]:
visual_10

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.042463 -0.332602       1        1  13.835261
1      0.328167  0.201502       2        1  13.499317
9     -0.232449  0.152157       3        1  11.941610
0     -0.000032 -0.021795       4        1   9.834746
3      0.010081 -0.034640       5        1   9.631570
5     -0.160118  0.100382       6        1   9.258376
8      0.001227 -0.020021       7        1   9.047059
4      0.003364 -0.020127       8        1   8.794635
6      0.008842 -0.015279       9        1   8.172894
7     -0.001544 -0.009576      10        1   5.984533, topic_info=          Term        Freq       Total Category  logprob  loglift
138       time  164.000000  164.000000  Default  30.0000  30.0000
162        bag   95.000000   95.000000  Default  29.0000  29.0000
40        hour  107.000000  107.000000  Default  28.0000  28.0000
95        seat   84.000000   84.000000  Default  27.0000  27.0000
7         gate  102.000000  102.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
1134  daughter    7.659693    8.550274  Topic10  -4.5955   2.7060
1106    boston    6.783279    7.673641  Topic10  -4.7170   2.6927
984       yeah    6.241085    7.131513  Topic10  -4.8003   2.6826
396    receive    5.991041    6.881662  Topic10  -4.8412   2.6774
1176   plastic    5.896166    6.786930  Topic10  -4.8572   2.6753

[332 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
185       8  0.914398  aadvantage
65       10  0.968589        able
531       3  0.979915       agent
186       8  0.960212         air
346       5  0.987278     airline
...     ...       ...         ...
103       2  0.955628        year
919       5  0.917421    year_old
129       4  0.968442         yes
534       9  0.948109   yesterday
561       1  0.968536         yet

[303 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 10, 1, 4, 6, 9, 5, 7, 8])

In [150]:
visual_20

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.333730  0.248459       1        1  8.448299
18    -0.106325 -0.237649       2        1  7.142758
10    -0.087394 -0.125877       3        1  6.952757
8     -0.103928 -0.161932       4        1  6.413356
19     0.025066  0.010549       5        1  6.369082
16    -0.006787 -0.009830       6        1  6.320390
13    -0.081754 -0.102243       7        1  5.736973
6      0.027660  0.014361       8        1  5.562920
14     0.055478  0.030294       9        1  5.292446
11     0.062118  0.033458      10        1  4.833535
3      0.053485  0.029476      11        1  4.741590
17     0.033765  0.019580      12        1  4.699545
4      0.042142  0.023806      13        1  4.391183
1      0.054515  0.029915      14        1  4.286768
7      0.056457  0.031053      15        1  3.926283
0      0.058378  0.031836      16        1  3.786570
9      0.054929  0.030287      17        1  3.744947
2      0.065858  0.035150      18        1  3.218247
5      0.067249  0.035702      19        1  3.195911
12     0.062820  0.033604      20        1  0.936440, topic_info=      Term        Freq       Total Category  logprob  loglift
138   time  251.000000  251.000000  Default  30.0000  30.0000
40    hour  181.000000  181.000000  Default  29.0000  29.0000
7     gate  155.000000  155.000000  Default  28.0000  28.0000
162    bag  143.000000  143.000000  Default  27.0000  27.0000
95    seat  165.000000  165.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
776   sure    0.059305    1.616131  Topic20  -7.6017   1.3657
555   plan    0.059301    1.615915  Topic20  -7.6018   1.3658
569    min    0.059303    1.616035  Topic20  -7.6017   1.3658
465  extra    0.059300    1.615875  Topic20  -7.6018   1.3658
406    day    0.059302    1.616880  Topic20  -7.6017   1.3652

[714 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
185      10  0.939231  aadvantage
65       10  0.963259        able
341      16  0.723438   according
73        9  0.826391    addition
47        7  0.849301  additional
...     ...       ...         ...
286       8  0.938522     working
381      16  0.818473       world
155       3  0.976050       would
103      15  0.959405        year
129      13  0.972120         yes

[381 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 19, 11, 9, 20, 17, 14, 7, 15, 12, 4, 18, 5, 2, 8, 1, 10, 3, 6, 13])

In [151]:
visual_30

TypeError: Object of type 'complex' is not JSON serializable

PreparedData(topic_coordinates=                        x                   y  topics  cluster      Freq
topic                                                                   
1      0.159718+0.000000j -0.151108+0.000000j       1        1  6.426750
17     0.140193+0.000000j -0.042821+0.000000j       2        1  5.853745
8      0.165076+0.000000j  0.413238+0.000000j       3        1  5.548140
2      0.152620+0.000000j -0.071950+0.000000j       4        1  5.498083
33     0.126230+0.000000j -0.022555+0.000000j       5        1  5.252715
11     0.125943+0.000000j -0.021159+0.000000j       6        1  5.097377
4      0.138085+0.000000j -0.027897+0.000000j       7        1  4.965352
27     0.140522+0.000000j -0.030423+0.000000j       8        1  4.911038
18     0.086083+0.000000j -0.008124+0.000000j       9        1  4.199463
22     0.121913+0.000000j -0.017774+0.000000j      10        1  4.138880
26     0.107921+0.000000j -0.013184+0.000000j      11        1  3.888603
29     0.121449+0.00

In [ ]:
visual_40

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
30     0.068108 -0.039195       1        1  3.682513
24     0.014063 -0.018936       2        1  3.415885
19     0.051371 -0.030565       3        1  3.252959
23     0.025983  0.091998       4        1  3.181579
36     0.045314  0.044267       5        1  3.115212
11     0.066605  0.010360       6        1  2.896816
14     0.008733 -0.018783       7        1  2.883337
0      0.066630  0.001202       8        1  2.746364
12    -0.015280  0.085314       9        1  2.745986
13    -0.029052  0.023876      10        1  2.675961
4     -0.084929 -0.028016      11        1  2.667583
1      0.042203  0.009821      12        1  2.637075
2      0.029645 -0.071027      13        1  2.635137
33    -0.061880 -0.020470      14        1  2.598794
34     0.086876  0.027759      15        1  2.558341
18    -0.013692  0.083437      16        1  2.531852
31     0.006163  0.010539      17        1  2.527473
29     0.012394  0.036876      18        1  2.526064
5     -0.036212 -0.018981      19        1  2.485343
8      0.068109  0.015222      20        1  2.476761
16    -0.064158 -0.089035      21        1  2.464933
6      0.077115  0.085606      22        1  2.404055
38    -0.050915 -0.127059      23        1  2.400335
10    -0.043901 -0.019577      24        1  2.392418
27     0.011733 -0.007297      25        1  2.316526
20    -0.046338  0.012322      26        1  2.295830
22    -0.044610  0.085596      27        1  2.251637
9     -0.004353 -0.035535      28        1  2.228714
21    -0.067642  0.005526      29        1  2.194331
39    -0.161949 -0.039878      30        1  2.193190
17     0.053697  0.024972      31        1  2.188314
3     -0.082468  0.075588      32        1  2.164537
26     0.035050 -0.111653      33        1  2.160973
35    -0.052004  0.077286      34        1  2.135766
15     0.036946  0.043199      35        1  2.107276
32    -0.036849 -0.036889      36        1  2.022164
28     0.066707 -0.013543      37        1  1.983071
37    -0.071353  0.055090      38        1  1.976376
7     -0.008290 -0.069596      39        1  1.975944
25     0.102427 -0.109821      40        1  1.902575, topic_info=      Term       Freq       Total Category  logprob  loglift
95    seat  98.000000   98.000000  Default  30.0000  30.0000
21   great  64.000000   64.000000  Default  29.0000  29.0000
162    bag  88.000000   88.000000  Default  28.0000  28.0000
180    way  56.000000   56.000000  Default  27.0000  27.0000
7     gate  97.000000   97.000000  Default  26.0000  26.0000
..     ...        ...         ...      ...      ...      ...
138   time   2.017391  111.434156  Topic40  -4.7837  -0.0497
95    seat   2.017383   98.614316  Topic40  -4.7837   0.0725
94     one   2.017382   62.715183  Topic40  -4.7837   0.5252
20    back   2.017353   60.033917  Topic40  -4.7837   0.5688
40    hour   2.017292   98.200117  Topic40  -4.7837   0.0767

[2523 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
185       1  0.273358  aadvantage
185       3  0.091119  aadvantage
185       5  0.091119  aadvantage
185       6  0.091119  aadvantage
185      15  0.091119  aadvantage
...     ...       ...         ...
976       7  0.132231       young
976      10  0.132231       young
976      19  0.132231       young
976      20  0.264463       young
976      33  0.264463       young

[7721 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[31, 25, 20, 24, 37, 12, 15, 1, 13, 14, 5, 2, 3, 34, 35, 19, 32, 30, 6, 9, 17, 7, 39, 11, 28, 21, 23, 10, 22, 40, 18, 4, 27, 36, 16, 33, 29, 38, 8, 26])

In [ ]:
visual_80

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
36     0.025407 -0.023719       1        1  2.583021
24     0.012327  0.077148       2        1  1.910146
34     0.106222 -0.020163       3        1  1.801997
19    -0.017145  0.053252       4        1  1.746362
0      0.034371  0.015626       5        1  1.710331
...         ...       ...     ...      ...       ...
73     0.078852  0.097552      76        1  0.850964
62     0.048738 -0.025022      77        1  0.832110
11     0.084143 -0.020621      78        1  0.795873
66    -0.033208 -0.010599      79        1  0.768655
61     0.016758 -0.003225      80        1  0.625392

[80 rows x 5 columns], topic_info=                 Term        Freq       Total Category  logprob  loglift
40               hour  101.000000  101.000000  Default  30.0000  30.0000
95               seat  100.000000  100.000000  Default  29.0000  29.0000
85                amp   78.000000   78.000000  Default  28.0000  28.0000
237             delay   54.000000   54.000000  Default  27.0000  27.0000
7                gate   96.000000   96.000000  Default  26.0000  26.0000
..                ...         ...         ...      ...      ...      ...
281              good    1.283207   39.386888  Topic80  -4.1236   1.6505
346           airline    1.283315   45.273190  Topic80  -4.1235   1.5113
114  customer_service    1.283274   50.980570  Topic80  -4.1235   1.3925
85                amp    1.283271   78.066183  Topic80  -4.1235   0.9664
42              still    1.283233   66.172059  Topic80  -4.1235   1.1317

[4211 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
185       3  0.081581  aadvantage
185       5  0.081581  aadvantage
185      20  0.163162  aadvantage
185      22  0.081581  aadvantage
185      27  0.081581  aadvantage
...     ...       ...         ...
976      11  0.123262       young
976      12  0.246523       young
976      15  0.123262       young
976      37  0.123262       young
976      56  0.123262       young

[9274 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[37, 25, 35, 20, 1, 71, 17, 39, 40, 76, 11, 49, 33, 28, 30, 48, 75, 18, 22, 13, 61, 21, 52, 15, 64, 66, 53, 51, 5, 43, 24, 59, 70, 57, 45, 36, 16, 60, 79, 41, 31, 50, 78, 8, 32, 55, 26, 47, 68, 34, 72, 7, 56, 29, 58, 2, 14, 10, 3, 46, 38, 19, 54, 27, 23, 44, 80, 77, 6, 9, 42, 69, 4, 73, 65, 74, 63, 12, 67, 62])

The less it overlaps and the better is the distribution into topics

#### Model coherence

In [ ]:
# lda model's coherence score
from gensim.models.coherencemodel import CoherenceModel

coherence_model_lda_10 = CoherenceModel(model=lda_model_10, texts=data['question_lem_bigrams'], dictionary=id2word, coherence='c_v')
coherence_model_lda_20 = CoherenceModel(model=lda_model_20, texts=data['question_lem_bigrams'], dictionary=id2word, coherence='c_v')
coherence_model_lda_30 = CoherenceModel(model=lda_model_30, texts=data['question_lem_bigrams'], dictionary=id2word, coherence='c_v')
coherence_model_lda_40 = CoherenceModel(model=lda_model_40, texts=data['question_lem_bigrams'], dictionary=id2word, coherence='c_v')
coherence_model_lda_80 = CoherenceModel(model=lda_model_80, texts=data['question_lem_bigrams'], dictionary=id2word, coherence='c_v')
coherence_lda_10 = coherence_model_lda_10.get_coherence()
coherence_lda_20 = coherence_model_lda_20.get_coherence()
coherence_lda_30 = coherence_model_lda_30.get_coherence()
coherence_lda_40 = coherence_model_lda_40.get_coherence()
coherence_lda_80 = coherence_model_lda_80.get_coherence()
print('Coherence Score_10: ', coherence_lda_10)
print('Coherence Score_20: ', coherence_lda_20)
print('Coherence Score_30: ', coherence_lda_30)
print('Coherence Score_40: ', coherence_lda_40)
print('Coherence Score_80: ', coherence_lda_80)

Coherence Score_10:  0.3560196605818572
Coherence Score_20:  0.33844339132929446
Coherence Score_30:  0.39108349319320007
Coherence Score_40:  0.4204914545790147
Coherence Score_80:  0.4572377567542064


more topics is more coherent but we must pick a good fit between coherence and a number of topics where classification is good enough

#### Try to extract topic from new text 

In [ ]:
from operator import itemgetter

In [ ]:
def preprocess(text):
    text_cleaned = gensim.utils.simple_preprocess(question_preprocess(text))
    text_lem = bigram_mod[lemmat(" ".join([w for w in text_cleaned if w not in stop_words]))]
    return text_lem

In [ ]:
def get_highest_topic(text):
  text_preprocessed = preprocess(text)
  result = lda_model_30[id2word.doc2bow(text_preprocessed)]
  return max(result, key=itemgetter(1))[0]

get_highest_topic('@AmericanAir terrible service wait ages trying to call customer service almost two months gone no response')

20

In [ ]:
data['topic'] = data['question'].apply(get_highest_topic)
data.head(10)

,responce,question,question_lem_bigrams,topic
0,@115904 We'll be sure to pass along your kind words! #AATeam,@AmericanAir Erica on the lax team is amazing give her a raise ty,"[erica, lax, team, amazing, give, raise]",23
1,@115904 Our apologies for the delay in responding to you. Have you made it to LAX? Let us know if you still need assistance.,@AmericanAir Could you have someone on your lax team available to guide me to my gate ASAP,"[could, someone, lax, team, available, guide, gate, asap]",20
2,"@115905 Aww, that's definitely a future pilot in the making! #HappyHalloween",Ben Tennyson and an American Airlines pilot. 🎃 #trunkortreat #halloween #2017 #diycostume #parenting @americanair … https://t.co/f1nNHQ0iLa https://t.co/lDViDkRdB1,"[ben, tennyson, american_airline, pilot]",0
3,@115906 We're sorry for your frustration.,"@AmericanAir Right, but I earned those. I also shouldn’t have to pay to pass them to my own spouse. You need to change your program.","[right, earned, also, pay, pas, spouse, need, change, program]",16
4,@115909 We're glad you got to kick back and enjoy a show while flying! Thanks for your kind words.,"Thank you, @AmericanAir for playing #ThisIsUs and for having great flight attendants on my flight back home!","[playing, great, attendant, back, home]",19
5,@116142 We never want your experience to be anything less than perfect. We're sorry the Wi-Fi was slower than expected.,@AmericanAir's wifi makes Amtrak's wifi look pro. At least theirs is free. @5804 has decent wifi @ reasonable prices. Why not AA?,"[wifi, make, amtrak, wifi, look, pro, least, free, decent, wifi, reasonable, price]",6
6,"@116143 Thanks for the shout-out and for stopping by, Marc! We'll pass this on to our SFO Admirals Club managers.","Wonderful club! @americanair (@ American Airlines Admirals Club in San Francisco, CA) https://t.co/GWFwuGVgNA","[wonderful, club, american_airline, admiral_club, san, francisco]",20
7,@116144 They'll be more than happy to walk you through it and help with the booking process.,"@AmericanAir already did...changed browsers, did all my techie tricks and still can't make a reservation after 1 hour!!","[already, changed, browser, techie, trick, still, make, reservation, hour]",4
8,@116144 We're sorry for the long wait. The next available agent will be with you as soon as possible.,@AmericanAir ........still....on....hold.....they made the booking. I am trying to pay.....so #frustrating.,"[still, hold, made, booking, trying, pay]",9
9,@116144 Fares sell in real time. We're sorry we weren't able to snag the lower fare for you.,@AmericanAir well now i am told the ticket cost is an additional $200/person to pay....because of the wait....omg,"[well, told, ticket, cost, additional, person, pay, wait, omg]",4


#### Building Intents

In [ ]:
#custom intents
intents = {"Greeting": ["Hello", "How are you doing?", "Greetings!", "How do you do?"],
          "Apology": ["No problem"],
          "Thanks": ["No problem", "You're welcome"],
          "Goodbye": ["It was nice speaking to you", "See you later", "Speak soon!"]
             }

In [ ]:
def update_intents(df):
  for index, row in df.iterrows():
    if str(row['topic']) not in intents :
      intents[str(row['topic'])] = []
    if row['responce'] != "":
      intents[str(row['topic'])].append(str(row['responce']))

In [ ]:
update_intents(data)

Save Intents in order to use it again

In [ ]:
with open('intents.json', 'w') as fp:
    json.dump(intents, fp)

#### Doc2Vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(data['responce'])]
model = Doc2Vec(tagged_data, window=1, min_count=2, epochs = 200)

#### First Results 

In [ ]:
def chatbot(text):
  result = model.dv.most_similar(positive=[model.infer_vector(preprocess(text))], topn=1850)
  topic = get_highest_topic(text)
  for l in result:
    if data['topic'].iloc[int(l[0])] == topic:
      return data['responce'].iloc[int(l[0])]

In [ ]:
#Preexisting questions and responses testing

list_word = ["@AmericanAir what’s going on with flight 301 DTW &gt; DFW? Delayed over 3 hours. Will it be canceled?", 
             "@AmericanAir awful service, more than 1 hour delay due to logbook maintenance issues. Where is preventive maintenance?",
             "@AmericanAir Yes. Refund my plane ticket😡 with your racist ass workers",
             "@AmericanAir I'm aware. Fits in overhead.This was a return trip. Had same luggage going. This was pre-security. Gate chck, ok, but Empty overheads on flt",
             "Thank you @AmericanAir for destroying not one but BOTH pieces of luggage we checked with you. Paid a checked baggage fee so our luggage could be ruined. Never again."]

list_resp = ["Looks like it has a maintenance delay and it's currently scheduled to take off at 10:10p.",
             "We do many checks pre and post departure. Please share your flight number if we can provide an update.",
             "We don't tolerate discrimination of any kind. Please DM your record locator and contact details (phone and email)",
             "We'd like to share this feedback. Please DM your record locator.",
             "We're sorry about that, Tyler. Please take your bags to the Baggage Service Office to file a claim."]

for q,r in zip(list_word, list_resp):
  print("QUESTION :", q)
  print("EXPECTED :", r)
  print("PREDICTED :", chatbot(q),"\n")
  


QUESTION : @AmericanAir what’s going on with flight 301 DTW &gt; DFW? Delayed over 3 hours. Will it be canceled?
EXPECTED : Looks like it has a maintenance delay and it's currently scheduled to take off at 10:10p.
PREDICTED : @168387 We'll have you at your gate and on your way as soon as we can. Thanks for your patience. 

QUESTION : @AmericanAir awful service, more than 1 hour delay due to logbook maintenance issues. Where is preventive maintenance?
EXPECTED : We do many checks pre and post departure. Please share your flight number if we can provide an update.
PREDICTED : @136930 It applies to 4 companions in the same reservation traveling within the 50 US, Canada, Puerto Rico and https://t.co/1iva8gJ6HM Islands. 

QUESTION : @AmericanAir Yes. Refund my plane ticket😡 with your racist ass workers
EXPECTED : We don't tolerate discrimination of any kind. Please DM your record locator and contact details (phone and email)
PREDICTED : @124728 That's not what we like to hear. We'll pass th

In [ ]:
while True : 
  question = input('Enter your question (exit to escape): ')
  if question.lower() == 'exit':
    break
  print("Question : ", question)
  print("           ---->", chatbot(question))